![ine-divider](https://user-images.githubusercontent.com/7065401/92672068-398e8080-f2ee-11ea-82d6-ad53f7feb5c0.png)
<hr>

### Secure RESTful APIs using Python

# SSL/TLS encryption

This project consists of creating secured connections between a service and a consumer.

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 1

**Generating unsigned keys**

For this task, you simply need to generate PEM keys as was shown in the lesson, and create a web service simlar to the one in the lesson.  A "hello world" response at any particular route is adequate for this server.  Once you can communicate with your service using an `https://...` URL, you have completed the task.

If you are on a Linux system, installing OpenSSL is straightforward using the package manager you are accustomed to.  On Mac OSX, the (Homebrew installer)[https://brew.sh/] provides similar capabilities to `apt-get`, `dnf`, or `zypper`.  On Windows installation is more difficult, but some binaries have been packaged at https://wiki.openssl.org/index.php/Binaries.  Building from source code on Windows is more secure, if you are able to (it's not difficult, but you will need a compiler toolchain).

Once you have installed OpenSSL and generated keys—e.g. `student-private.pem` and `student-public.pem`—create a small server that will correctly respond to:

```python
import requests
resp = requests.get("https://localhost:5000/hello", verify='student-public.pem')
assert resp.status_code == 200
```

**A possible solution**

For Linux, the lesson gives basically the exact solution.  Generate keys with:

```
openssl req -x509 -newkey rsa:4096 -nodes -out student-public.pem -keyout student-private.pem -days 30
```

Once keys exist, a server can simply consist of:

```python
#!/usr/bin/env python
from flask import Flask
app = Flask(__name__)

@app.route("/hello")
def hello():
    return "Hello world"

if __name__ == "__main__":
    app.run(ssl_context=('student-public.pem', 'student-private.pem'))
```

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)

## Part 2

**Obtain Certificate Authority signed keys**

For long term use, obtaining private/public key pairs from a signed authority is good practice.  If you are able to, for this task generate CA signed keys and install them on the domain they are signed for.

A list of Certificate Authorities can be found on Wikipedia: https://en.wikipedia.org/wiki/Certificate_authority  The non-profit [Let's Encrypt](https://letsencrypt.org/) provides certificates free-of-cost, and has tools for automatic generation of certificates.  The exact procedure you will use will vary between CAs.

If you succeed in this task, you will be able to connect to the "Hello world" service in the first task without seeing a warning or needing to run:

```python
requests.packages.urllib3.disable_warnings()
```

**A possible solution**

The solution varies greatly by chosen CA and by web server and operating system you use.  Moreover, for a CA signed certificate to work, you will need to host your service on a domain name or public IP address to issue a key.

Let's Encrypt provides some helpful advice for application or micro-service development on `localhost` at:

> https://letsencrypt.org/docs/certificates-for-localhost/

The main step involved is creating self-signed keys using a command similar to:

```
openssl req -x509 -out localhost.crt -keyout localhost.key \
  -newkey rsa:2048 -nodes -sha256 \
  -subj '/CN=localhost' -extensions EXT -config <( \
   printf "[dn]\nCN=localhost\n[req]\ndistinguished_name = dn\n[EXT]\nsubjectAltName=DNS:localhost\nkeyUsage=digitalSignature\nextendedKeyUsage=serverAuth")
```

![orange-divider](https://user-images.githubusercontent.com/7065401/92672455-187a5f80-f2ef-11ea-890c-40be9474f7b7.png)